# Padé appproximation

In [1]:
import logging

import numpy

import cicada
from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator

from statistics import mean, stdev
from tqdm import *

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = AdditiveProtocolSuite(communicator)

    values = numpy.array(1)
    log.info(f"Player {communicator.rank} values: {values}")

    values_share = protocol.share(src=0, secret=values, shape=())
    pade_tanh_share = protocol._pade_approx(numpy.tanh, values_share)
    pade_tanh = pade_tanh_share
    log.info(f"Player {communicator.rank} pade_tanh({values}): {pade_tanh} vs tanh({values}): {numpy.tanh(values)}")
    abs_err_list = []
    for i in tqdm(numpy.arange(-1, 1, .1)):
        cic_pade_tanh = protocol._pade_approx(numpy.tanh, protocol.share(src=0, secret=numpy.array(i), shape=()))
        legit_tanh = numpy.tanh(i)
        abs_err_list.append(abs(legit_tanh-cic_pade_tanh))
    

    
    if communicator.rank ==1:
        print(mean(abs_err_list), stdev(abs_err_list))
        print(pn(1)/pd(1))

SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 values: 1
INFO:root:Player 1 values: 1
INFO:root:Player 2 values: 1
INFO:root:Player 0 pade_tanh(1): -0.00532910190130071 vs tanh(1): 0.7615941559557649
INFO:root:Player 1 pade_tanh(1): -0.00532910190130071 vs tanh(1): 0.7615941559557649
INFO:root:Player 2 pade_tanh(1): -0.00532910190130071 vs tanh(1): 0.7615941559557649
100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.03it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.02it/s]


0.43669584918135845 0.23378196059971137


ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 1 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/home/kgoss/git/cicada-mpc/cicada/communicator/socket/__init__.py", line 784, in launch
    result = fn(communicator, *args, **kwargs)
  File "/tmp/ipykernel_12294/2734818129.py", line 35, in main
    print(pn(1)/pd(1))
NameError: name 'pn' is not defined



In [3]:
from scipy.interpolate import approximate_taylor_polynomial, pade
degree = 9
func_taylor = approximate_taylor_polynomial(numpy.tanh, 0, degree, 1)
paden, paded = pade(func_taylor, degree//2, degree%2+degree//2)
print(paden(1)/paded(1))

-0.00533113419753974
